confusion matrix :
    http://gcucurull.github.io/caffe/python/deep-learning/2016/06/29/caffe-confusion-matrix/
    https://github.com/gcucurull/caffe-conf-matrix
FCN :
    score :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/score.py
    infer :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/infer.py

In [ ]:
from __future__ import division
import caffe
import numpy as np
import os
import sys
from datetime import datetime
from PIL import Image

In [ ]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

In [ ]:
import _init_paths
import caffe
#import jg_layers.jg_input_voc_layers

In [ ]:
def eval(hist,loss, it):
    print '>>>', datetime.now(), 'Iteration', it, 'loss', loss
    # overall accuracy
    acc = np.diag(hist).sum() / hist.sum()
    print '>>>', datetime.now(), 'Iteration', it, 'overall accuracy', acc
    # per-class accuracy
    acc = np.diag(hist) / hist.sum(1)
    print '>>>', datetime.now(), 'Iteration', it, 'mean accuracy', np.nanmean(acc)
    # per-class IU
    iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))
    print '>>>', datetime.now(), 'Iteration', it, 'mean IU', np.nanmean(iu)
    freq = hist.sum(1) / hist.sum()
    print '>>>', datetime.now(), 'Iteration', it, 'fwavacc', (freq[freq > 0] * iu[freq > 0]).sum()

In [ ]:
model = 'rgb_100'
newParams = {}

#classes
newParams['classes_file_path']= '/home/jogue/workspace/datasets/SUNRGBD_pv/37_classes.txt'

# images
newParams['images_folder']= '/home/jogue/workspace/datasets/SUNRGBD_pv/data_nyudv2/rgb_i_range_10-10-100/testGupta/'
newParams['image_file_extension']= 'png'
# segmentations
newParams['segmentations_folder']= '/home/jogue/workspace/datasets/SUNRGBD_pv/data_nyudv2/labels_segmentation_37d_range_10-10-100/testGupta/'                            
newParams['segmentation_file_extension']= 'png'

#sets
newParams['set_file_path']= '/home/jogue/workspace/datasets/SUNRGBD_pv/data/sets/nyudv2_gupta/testGupta_10-10-100.txt'

#shuffme
newParams['shuffle']= False

In [ ]:
ICIPmodels = '/home/jogue/workspace/ICIP/models/'
prototxt_path = ICIPmodels+model+'/jg_eval.prototxt'
weights_path = ICIPmodels+model+'/weights.caffemodel'
caffe_gpu = True


if !caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(gpu_id);

In [ ]:
net = caffe.Net(prototxt_path, caffe.TEST)
#net = caffe.Net(prototxt_path,weights_path , caffe.TEST)

#### Blobs & Layers

In [ ]:
B = [(k, v.data.shape) for k, v in net.blobs.items()]
L = [(k, v[0].data.shape, v[1].data.shape) for k, v in net.params.items() if k != 'upscore37']

#### Updating with new params

In [ ]:
net.layers[0].update(newParams)

In [ ]:
#hist, loss = compute_hist(net, save_format, dataset, layer, gt)
n_cl = net.blobs['score'].channels
hist = np.zeros((n_cl, n_cl))
loss = 0
for idx in range(net.layers[0].nb_images):
    print "Iter counter =",net.layers[0].iter_counter
    net.forward()
    hist += fast_hist(net.blobs['label'].data[0, 0].flatten(),net.blobs['score'].data[0].argmax(0).flatten(),n_cl)
    # compute the loss as well
    loss += net.blobs['loss'].data.flat[0]
    eval(hist,loss/net.layers[0].nb_images, net.layers[0].iter_counter)